In [1]:
!pip install --upgrade pythainlp
!pip install --upgrade pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 27.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
numba 0.56.4 requires numpy<

In [1]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
df=pd.read_csv('dataset_terminal21_pattaya_rev1.csv')

In [3]:
df.tail()

,Review_ID,Review
17,18,มันเป็นห้างสรรพสินค้าขนาดใหญ่ที่ทันสมัยแห่งใหม...
18,19,ท่องเที่ยวห้างสรรพสินค้าที่มีธีมพร้อมร้านอาหาร...
19,20,เราพบแบรนด์ที่แตกต่างกันมากทำเลดีมากแต่ละชั้นใ...
20,21,รูปแบบห้างสรรพสินค้าใหม่ที่ดีในเมืองร้านอาหารแ...
21,22,ห้างสรรพสินค้าขนาดใหญ่ที่มีร้านค้ามากมายหลังจา...


In [4]:
stopwords=list(pythainlp.corpus.thai_stopwords())
removed_words=['', ' ','  ','   ', '    ', '     ','\n','/','ว่า','ๆ', '(', ')','พัทยา','การ','มี','ร้าน','ที่','...', '2','-']
screening_words=stopwords + removed_words
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    merged = merged + ',' + word
  return merged[1:]

In [5]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [6]:
df.tail()

,Review_ID,Review,Review_tokenized
17,18,มันเป็นห้างสรรพสินค้าขนาดใหญ่ที่ทันสมัยแห่งใหม...,"มัน,เป็น,ห้างสรรพสินค้า,ขนาดใหญ่,ที่,ทันสมัย,แ..."
18,19,ท่องเที่ยวห้างสรรพสินค้าที่มีธีมพร้อมร้านอาหาร...,"ท่องเที่ยว,ห้างสรรพสินค้า,ที่,มี,ธีม,พร้อม,ร้า..."
19,20,เราพบแบรนด์ที่แตกต่างกันมากทำเลดีมากแต่ละชั้นใ...,"เรา,พบ,แบรนด์,ที่,แตก,ต่างกัน,มาก,ทำเล,ดีมาก,แ..."
20,21,รูปแบบห้างสรรพสินค้าใหม่ที่ดีในเมืองร้านอาหารแ...,"รูปแบบ,ห้างสรรพสินค้า,ใหม่,ที่,ดี,ใน,เมือง,ร้า..."
21,22,ห้างสรรพสินค้าขนาดใหญ่ที่มีร้านค้ามากมายหลังจา...,"ห้างสรรพสินค้า,ขนาดใหญ่,ที่,มี,ร้านค้า,มากมาย,..."


In [7]:
documents=df['Review_tokenized'].to_list()
texts=[[text for text in doc.split(',')] for doc in documents]
dictionary=gensim.corpora.Dictionary(texts)

In [8]:
print(dictionary.token2id.keys())

dict_keys(['ก\u200b', 'ของ', 'ความสุข', 'คาเฟ่', 'คุณ', 'ช่วย', 'ซุปเปอร์มาร์เก็ต', 'ทรอ', 'ทริป', 'ที่จะ', 'ท่องเที่ยว', 'นิ', 'พัทยา', 'มี', 'ร้านอาหาร', 'สนุกเกอร์', 'สินค้า', 'หลากหลาย', 'ห้างสรรพสินค้า', 'อาทิ', 'อิ', 'อุปกรณ์', 'เกมส์', 'เครื่องประดับ', 'เติมเต็ม', 'เล็ก', 'เสื้อผ้า', 'และ', 'โซน', 'ใน', 'ให้บริการ', '\u200b', ' ', 'กิน', 'ของกิน', 'คือ', 'ตกแต่ง', 'ตาม', 'ถูก', 'ถ่ายรูป', 'ที่', 'ฟู๊ด', 'มา', 'มาก', 'ราคา', 'สวยงาม', 'สำคัญ', 'ห้าง', 'อร่อย', 'อาหาร', 'เทศกาล', 'เยอะ', 'เหมาะสำหรับ', '21', 'centre', 'court', 'food', 'grand', 'pier', 'point', 'กว้างขวาง', 'ครบ', 'คุณภาพ', 'จบ', 'จอด', 'ถึง', 'ทุ่ม', 'พัก', 'ยาก', 'รถ', 'สมัยนี้', 'สะดวก', 'หา', 'ห้า', 'ออก', 'เดียว', 'เปิด', 'เลย', 'โรงหนัง', 'โรงแรม', 'ใหม่', 'ได้', 'ไป', 'ไม่ต้อง', 'ไหน', 'การ', 'จัด', 'ชั้น', 'ดีมาก', 'ต่างประเทศ', 'ถือได้ว่า', 'บรรยากาศ', 'ประเทศ', 'ภายใน', 'ร้านค้า', 'ลงตัว', 'สถานที่', 'สะดวกสบาย', 'ห้องน้ำ', 'เป็น', 'เมือง', 'เหมาะสม', 'แต่ละ', 'แบ่ง', 'แยก', 'โดยเฉพาะ', 'Court', 'Food', '

In [9]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies=[[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [12]:
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number fo passes through documents
iterations = 50
eval_every = 1 #Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary
temp = dictionary[0] # This is only to "load" the dicjjtionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize = chunksize, \
                                     alpha='auto', eta='auto', \
                                     iterations=iterations, num_topics=num_topics, \
                                     passes=passes, eval_every=eval_every)

CPU times: user 490 ms, sys: 1.69 ms, total: 492 ms
Wall time: 894 ms


In [13]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.084442 -0.016913       1        1  56.374869
1      0.059021 -0.055301       2        1  27.033922
2      0.025421  0.072214       3        1  16.591209, topic_info=         Term       Freq      Total Category  logprob  loglift
40        ที่  27.000000  27.000000  Default  30.0000  30.0000
31          ​  14.000000  14.000000  Default  29.0000  29.0000
29         ใน   7.000000   7.000000  Default  28.0000  28.0000
32             27.000000  27.000000  Default  27.0000  27.0000
151  ขนาดใหญ่   4.000000   4.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
9       ที่จะ   0.745190   3.398695   Topic3  -5.1998   0.2788
196     พร้อม   0.745181   2.724919   Topic3  -5.1998   0.4997
51       เยอะ   0.745131   7.454954   Topic3  -5.1999  -0.5068
166       มุม   0.745076   3.673533   Topic3  -5.1999   0.2009
109   กรุงเทพ   0.745014   2.862281   Topic3  -5.2000   0.4503

[203 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
32        1  0.585669      
32        2  0.366043      
32        3  0.036604      
240       3  0.806479     !
107       1  0.868180  Food
...     ...       ...   ...
81        2  0.253933   ได้
81        3  0.126967   ได้
143       1  0.844345   ไม่
172       3  0.806507   ไว้
31        1  0.967367     ​

[214 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [14]:
model.show_topic(1)

[(' ', 0.046937883),
 ('ที่', 0.028563881),
 ('และ', 0.022442844),
 ('มาก', 0.0193841),
 ('ห้าง', 0.01632601),
 ('ขนาดใหญ่', 0.016325785),
 ('มี', 0.013265176),
 ('อาหาร', 0.010207982),
 ('มา', 0.010205718),
 ('ได้', 0.010204427)]

In [15]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [16]:
df.tail()

,Review_ID,Review,Review_tokenized,topics,score
17,18,มันเป็นห้างสรรพสินค้าขนาดใหญ่ที่ทันสมัยแห่งใหม...,"มัน,เป็น,ห้างสรรพสินค้า,ขนาดใหญ่,ที่,ทันสมัย,แ...",1,0.998845
18,19,ท่องเที่ยวห้างสรรพสินค้าที่มีธีมพร้อมร้านอาหาร...,"ท่องเที่ยว,ห้างสรรพสินค้า,ที่,มี,ธีม,พร้อม,ร้า...",2,0.995005
19,20,เราพบแบรนด์ที่แตกต่างกันมากทำเลดีมากแต่ละชั้นใ...,"เรา,พบ,แบรนด์,ที่,แตก,ต่างกัน,มาก,ทำเล,ดีมาก,แ...",0,0.997042
20,21,รูปแบบห้างสรรพสินค้าใหม่ที่ดีในเมืองร้านอาหารแ...,"รูปแบบ,ห้างสรรพสินค้า,ใหม่,ที่,ดี,ใน,เมือง,ร้า...",2,0.996535
21,22,ห้างสรรพสินค้าขนาดใหญ่ที่มีร้านค้ามากมายหลังจา...,"ห้างสรรพสินค้า,ขนาดใหญ่,ที่,มี,ร้านค้า,มากมาย,...",1,0.997805
